# Chapter 11 Toolkit — Dimensionality Reduction (Lecture Notes pp. 164–178)

This notebook is a **complete, reusable toolkit** for Chapter 11:

## 11.1 Random Projection + Johnson–Lindenstrauss
- Random projection theorem (Thm 11.1): norm preservation for a fixed vector
- JL lemma (Thm 11.2): preserve all pairwise distances for n points
- Practical utilities: choose `k`, build random projection, measure distortion

## 11.2–11.3 SVD + PCA
- Singular vectors/values from `A^T A` (Lemma 11.7)
- Greedy best-fit k-subspace interpretation (Thm 11.10)
- SVD: `A = U D V^T`, PCA(A) = A V = U D (after centering)
- Power method for top eigenvectors / singular vectors

## 11.4 SVD in action
- Rank-k approximation `A_k`
- Reconstruction error and explained variance
- Simple anomaly detection via reconstruction error quantiles

## 11.5–11.6 Theory
- Empirical covariance concentration (Matrix Bernstein, Thm 11.15)
- Weyl’s theorem for eigenvalue stability (Thm 11.17)
- Excess reconstruction risk bound (Lemma 11.19, Thm 11.20)

> Run top-to-bottom once, then reuse the function sections by changing parameters (k, eps, d, n, C, etc.). ✅


In [ ]:
import numpy as np
import math
from dataclasses import dataclass
from typing import Callable, Tuple, Dict, Optional, List

import matplotlib.pyplot as plt


## 11.1 Random Projection (Thm 11.1) and JL Lemma (Thm 11.2)

### Random projection map
Given i.i.d. vectors \(U_1,...,U_k \in \mathbb{R}^d\), define:
\[
f(v) = (U_1\cdot v,\dots,U_k\cdot v)\in\mathbb{R}^k
\]

**Thm 11.1** (for sub-Gaussian components, variance \(a^2\)): for \(\|v\|=1\),
\[
\mathbb{P}\big(|\|f(v)\| - \sqrt{k}|a||v|| \ge \varepsilon \sqrt{k}|a||v|\big) \le 2e^{-k\varepsilon^2/128}
\]

**Thm 11.2 (JL)** for \(n\) points: if \(k > \frac{384\ln(n)}{\varepsilon^2}\), then w.h.p.
all pairwise distances are preserved up to factor \(1\pm \varepsilon\) (after dividing out \(\sqrt{k}\)).

Below are reusable functions that implement the algorithm and measure distortion.


In [ ]:
def random_projection_bound_thm11_1(k: int, eps: float) -> float:
    """Thm 11.1 tail bound: 2 exp(-k eps^2 / 128)."""
    if k <= 0 or not (0 < eps < 1):
        raise ValueError("k>0 and eps in (0,1).")
    return float(2.0 * math.exp(-k * eps * eps / 128.0))

def jl_required_k(n_points: int, eps: float) -> int:
    """Thm 11.2 requirement: k > 384 ln(n) / eps^2."""
    if n_points <= 1 or not (0 < eps < 1):
        raise ValueError("n_points>1 and eps in (0,1).")
    return int(math.floor(384.0 * math.log(n_points) / (eps * eps)) + 1)

def jl_success_prob_lower(n_points: int) -> float:
    """From Thm 11.2: success prob >= 1 - 3/(2n)."""
    if n_points <= 1:
        raise ValueError("n_points>1.")
    return float(1.0 - 3.0 / (2.0 * n_points))

def sample_subgaussian_matrix(d: int, k: int, a: float = 1.0, rng: Optional[np.random.Generator] = None) -> np.ndarray:
    """Practical choice: Gaussian N(0,a^2) entries are sub-Gaussian with parameter 1 (up to constants)."""
    rng = np.random.default_rng() if rng is None else rng
    return rng.normal(loc=0.0, scale=float(a), size=(k, d))

def random_projection_map(X: np.ndarray, R: np.ndarray, scale_by_sqrt_k: bool = True) -> np.ndarray:
    """Project points X (n,d) using matrix R (k,d). Returns (n,k)."""
    X = np.asarray(X, dtype=float)
    R = np.asarray(R, dtype=float)
    Y = X @ R.T
    if scale_by_sqrt_k:
        Y = Y / math.sqrt(R.shape[0])
    return Y

def pairwise_distances(X: np.ndarray) -> np.ndarray:
    """All-pairs Euclidean distances (n,n). O(n^2). Use for moderate n."""
    X = np.asarray(X, dtype=float)
    G = X @ X.T
    sq = np.maximum(np.diag(G)[:, None] - 2*G + np.diag(G)[None, :], 0.0)
    return np.sqrt(sq)

def relative_distance_errors(X: np.ndarray, Y: np.ndarray, eps_floor: float = 1e-12) -> np.ndarray:
    """Return relative errors |dY - dX| / dX for all i<j."""
    DX = pairwise_distances(X)
    DY = pairwise_distances(Y)
    n = DX.shape[0]
    errs = []
    for i in range(n):
        for j in range(i+1, n):
            denom = max(float(DX[i, j]), eps_floor)
            errs.append(abs(float(DY[i, j]) - float(DX[i, j])) / denom)
    return np.array(errs, dtype=float)


### Demo: JL distortion on synthetic data

This demo shows:
- choosing k using the JL formula
- projecting data
- plotting distribution of relative distance error

(For large n, pairwise distances are O(n²). Keep n moderate.)


In [ ]:
def demo_jl(n: int = 200, d: int = 800, eps: float = 0.25, a: float = 1.0, seed: int = 0):
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n, d))
    k = jl_required_k(n, eps)
    R = sample_subgaussian_matrix(d, k, a=a, rng=rng)
    Y = random_projection_map(X, R, scale_by_sqrt_k=True)

    errs = relative_distance_errors(X, Y)
    return {"n": n, "d": d, "k": k, "eps": eps, "success_prob_lb": jl_success_prob_lower(n), "errs": errs}

out = demo_jl(n=180, d=600, eps=0.30, seed=1)
print({k:v for k,v in out.items() if k != "errs"})
plt.figure()
plt.hist(out["errs"], bins=50)
plt.title("Relative distance error distribution after JL projection")
plt.xlabel("relative error")
plt.ylabel("count")
plt.show()
print("fraction within eps:", float(np.mean(out["errs"] <= out["eps"])))


## 11.2 SVD: singular vectors/values and the power method (Lemma 11.7 + Sec 11.2.1)

Given A (n×m), define first singular vector:
\[
v_1 = \arg\max_{\|v\|=1} \|A v\|,\quad \sigma_1 = \|A v_1\|.
\]

Lemma 11.7 says:
- \(v_1\) is the top eigenvector of \(A^T A\) with eigenvalue \(\lambda_1\)
- \(\sigma_1 = \sqrt{\lambda_1}\)

We implement:
- power method for top eigenvector of a symmetric matrix
- top-k eigenvectors (deflation)
- singular vectors/values from \(A^T A\)


In [ ]:
def power_method_top_eigenvector(
    B: np.ndarray,
    n_iter: int = 2000,
    tol: float = 1e-10,
    seed: int = 0,
) -> Tuple[np.ndarray, float, Dict[str, object]]:
    """Top eigenvector for symmetric B via power method. Returns (v, eigenvalue_est, info)."""
    B = np.asarray(B, dtype=float)
    if B.shape[0] != B.shape[1]:
        raise ValueError("B must be square.")
    # symmetric check is soft; allow small numerical asymmetry
    if not np.allclose(B, B.T, atol=1e-8):
        # symmetrize for safety
        B = 0.5*(B + B.T)

    rng = np.random.default_rng(seed)
    v = rng.normal(size=B.shape[0])
    v /= np.linalg.norm(v)

    prev = None
    for it in range(n_iter):
        w = B @ v
        normw = np.linalg.norm(w)
        if normw == 0:
            raise RuntimeError("Power method hit zero vector; B may be zero.")
        v_new = w / normw
        if prev is not None and np.linalg.norm(v_new - prev) < tol:
            v = v_new
            break
        prev = v_new
        v = v_new

    eig = float(v @ (B @ v))
    return v, eig, {"iters": it+1, "eig_est": eig}

def top_k_eigenvectors_deflation(B: np.ndarray, k: int, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    """Compute top-k eigenvectors/values of symmetric B using power method + deflation."""
    B = np.asarray(B, dtype=float)
    if not np.allclose(B, B.T, atol=1e-8):
        B = 0.5*(B + B.T)
    n = B.shape[0]
    V = np.zeros((n, k), dtype=float)
    vals = np.zeros(k, dtype=float)
    B_work = B.copy()

    for j in range(k):
        v, eig, _ = power_method_top_eigenvector(B_work, seed=seed+j)
        V[:, j] = v
        vals[j] = eig
        # Deflate: B <- B - eig * v v^T
        B_work = B_work - eig * np.outer(v, v)
    return V, vals

def svd_from_ata(A: np.ndarray, k: Optional[int] = None, seed: int = 0) -> Dict[str, np.ndarray]:
    """Compute right singular vectors/values via eigen-decomposition of A^T A."""
    A = np.asarray(A, dtype=float)
    B = A.T @ A  # (m,m)
    m = B.shape[0]
    if k is None:
        k = m
    k = min(k, m)

    # For robustness, use numpy eig for full; power method for large m
    if k == m and m <= 400:
        eigvals, eigvecs = np.linalg.eigh(B)
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        V = eigvecs[:, order]
    else:
        V, eigvals = top_k_eigenvectors_deflation(B, k=k, seed=seed)

    sigmas = np.sqrt(np.maximum(eigvals[:k], 0.0))
    V = V[:, :k]
    # Left singular vectors: u_i = A v_i / sigma_i (if sigma_i > 0)
    U = np.zeros((A.shape[0], k), dtype=float)
    for i in range(k):
        if sigmas[i] > 1e-12:
            U[:, i] = (A @ V[:, i]) / sigmas[i]
    return {"U": U, "S": sigmas, "V": V}


### Demo: SVD via numpy vs via \(A^T A\)

We compare top singular values and directions.


In [ ]:
rng = np.random.default_rng(0)
A = rng.normal(size=(300, 40))
res = svd_from_ata(A, k=6, seed=0)

U_np, S_np, Vt_np = np.linalg.svd(A, full_matrices=False)
print("Top-6 sigma (A^T A):", np.round(res["S"][:6], 6))
print("Top-6 sigma (numpy):", np.round(S_np[:6], 6))


## 11.3 PCA (coordinate change after centering)

The notes assume the data matrix A has **empirical mean 0** (centered).

If `A = U D V^T`, then PCA coordinates are:
\[
\mathrm{PCA}(A) = A V = U D.
\]

So PCA is: **project onto right singular vectors** (columns of V).


In [ ]:
def center_data(X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Center rows of X (n,d). Returns (X_centered, mean_vector)."""
    X = np.asarray(X, dtype=float)
    mu = np.mean(X, axis=0, keepdims=True)
    return X - mu, mu.squeeze()

def pca_fit(X: np.ndarray, k: int) -> Dict[str, np.ndarray]:
    """Fit PCA using SVD. Returns components (V_k), singular values, mean, scores."""
    Xc, mu = center_data(X)
    U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
    V = Vt.T
    Vk = V[:, :k]
    scores = Xc @ Vk  # (n,k)
    return {"mean": mu, "components": Vk, "singular_values": S, "scores": scores}

def pca_transform(X: np.ndarray, pca: Dict[str, np.ndarray]) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    Xc = X - pca["mean"]
    return Xc @ pca["components"]

def pca_inverse_transform(Z: np.ndarray, pca: Dict[str, np.ndarray]) -> np.ndarray:
    Z = np.asarray(Z, dtype=float)
    return Z @ pca["components"].T + pca["mean"]


## 11.4 Rank-k approximation, reconstruction error, explained variance

Given SVD: \(A=\sum_{j=1}^m \sigma_j u_j v_j^T\). Define rank-k approx:
\[
A_k = \sum_{j=1}^k \sigma_j u_j v_j^T.
\]

Reconstruction error (Frobenius):
\[
\|A-A_k\|_F = \sqrt{\sum_{j=k+1}^m \sigma_j^2}.
\]

Explained variance ratio by first k components:
\[
\frac{\sum_{j=1}^k \sigma_j^2}{\sum_{j=1}^m \sigma_j^2}.
\]


In [ ]:
def rank_k_approximation(X: np.ndarray, k: int) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
    """Return X_k rank-k reconstruction using SVD on centered data? (caller decides)."""
    X = np.asarray(X, dtype=float)
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    Uk = U[:, :k]
    Sk = S[:k]
    Vtk = Vt[:k, :]
    Xk = (Uk * Sk) @ Vtk
    return Xk, {"U": U, "S": S, "Vt": Vt}

def reconstruction_error_frobenius(X: np.ndarray, Xk: np.ndarray) -> float:
    D = np.asarray(X, dtype=float) - np.asarray(Xk, dtype=float)
    return float(np.linalg.norm(D, ord='fro'))

def reconstruction_error_from_singular_values(S: np.ndarray, k: int) -> float:
    S = np.asarray(S, dtype=float)
    tail = S[k:]
    return float(np.sqrt(np.sum(tail * tail)))

def explained_variance_ratio_from_singular_values(S: np.ndarray, k: int) -> float:
    S = np.asarray(S, dtype=float)
    num = float(np.sum(S[:k] * S[:k]))
    den = float(np.sum(S * S)) if np.sum(S*S) > 0 else float("nan")
    return num / den

# Demo: compress synthetic data
rng = np.random.default_rng(0)
X = rng.normal(size=(500, 60))
Xc, _ = center_data(X)

X10, svd_info = rank_k_approximation(Xc, k=10)
print("Recon error (fro):", reconstruction_error_frobenius(Xc, X10))
print("Recon error (sigma tail):", reconstruction_error_from_singular_values(svd_info["S"], k=10))
print("Explained variance ratio k=10:", explained_variance_ratio_from_singular_values(svd_info["S"], k=10))


### Optional demo: digits compression (like the notes' MNIST example)

The notes mention MNIST; offline we can use scikit-learn's `load_digits` dataset (8×8 images).
If scikit-learn isn't available in your environment, you can skip this cell safely.


In [ ]:
# Optional: digits dataset compression (offline, small)
try:
    from sklearn.datasets import load_digits
    digits = load_digits()
    A = digits.data.astype(float)  # (n,64)
    A_centered, mu = center_data(A)

    k = 10
    Ak, svd_info = rank_k_approximation(A_centered, k=k)
    recon = Ak + mu

    # Show 10 original vs reconstructed
    idx = np.arange(10)
    plt.figure(figsize=(8, 2))
    for i, j in enumerate(idx):
        plt.subplot(2, 10, i+1)
        plt.imshow(digits.images[j], cmap='gray')
        plt.axis('off')
        plt.subplot(2, 10, 10+i+1)
        plt.imshow(recon[j].reshape(8,8), cmap='gray')
        plt.axis('off')
    plt.suptitle("Top row: original | Bottom row: rank-10 reconstruction")
    plt.show()

    # Explained variance
    evr10 = explained_variance_ratio_from_singular_values(svd_info["S"], k=10)
    print("Explained variance ratio (k=10):", evr10)
except Exception as e:
    print("Skipped digits demo (likely sklearn missing):", repr(e))


## 11.4.3 Anomaly detection via reconstruction error

Procedure (as described):
1) Fit PCA/SVD on normal training data, choose k
2) Reconstruct each training point and compute reconstruction error
3) Choose a quantile threshold (e.g., 0.99)
4) Flag test points with reconstruction error above threshold

Below are reusable helpers.


In [ ]:
def reconstruction_errors_per_row(X: np.ndarray, Xk: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    Xk = np.asarray(Xk, dtype=float)
    return np.linalg.norm(X - Xk, axis=1)

def anomaly_threshold_from_quantile(errors: np.ndarray, q: float = 0.99) -> float:
    return float(np.quantile(np.asarray(errors, dtype=float), q))

def pca_anomaly_detector_fit(X_train: np.ndarray, k: int, q: float = 0.99) -> Dict[str, object]:
    pca = pca_fit(X_train, k=k)
    Z = pca_transform(X_train, pca)
    X_rec = pca_inverse_transform(Z, pca)
    errs = reconstruction_errors_per_row(X_train, X_rec)
    thr = anomaly_threshold_from_quantile(errs, q=q)
    return {"pca": pca, "k": k, "q": q, "threshold": thr, "train_errors": errs}

def pca_anomaly_detector_predict(det: Dict[str, object], X: np.ndarray) -> Dict[str, np.ndarray]:
    pca = det["pca"]
    Z = pca_transform(X, pca)
    X_rec = pca_inverse_transform(Z, pca)
    errs = reconstruction_errors_per_row(X, X_rec)
    flags = errs > det["threshold"]
    return {"errors": errs, "is_anomaly": flags, "threshold": float(det["threshold"])}

# Demo: create anomalies by adding heavy noise to some points
rng = np.random.default_rng(0)
X_train = rng.normal(size=(800, 40))
X_test = rng.normal(size=(300, 40))
X_test[:20] += rng.normal(scale=6.0, size=(20, 40))  # inject anomalies

det = pca_anomaly_detector_fit(X_train, k=5, q=0.99)
pred = pca_anomaly_detector_predict(det, X_test)

print("threshold:", pred["threshold"])
print("flagged anomalies:", int(np.sum(pred["is_anomaly"])))
print("flagged among injected 20:", int(np.sum(pred["is_anomaly"][:20])))

plt.figure()
plt.hist(det["train_errors"], bins=50, alpha=0.7, label="train errors")
plt.hist(pred["errors"], bins=50, alpha=0.5, label="test errors")
plt.axvline(pred["threshold"], linestyle='--', label="threshold")
plt.title("Reconstruction errors for anomaly detection")
plt.xlabel("||x - x_rec||")
plt.ylabel("count")
plt.legend()
plt.show()


## 11.5 Theory: empirical covariance concentration (Thm 11.15) + Weyl (Thm 11.17)

Empirical covariance (centered data):
\[
\hat\Sigma = \frac1n\sum_{i=1}^n X_i X_i^T.
\]

**Matrix Bernstein** (Thm 11.15): if \(\|X_i\|_2 \le \sqrt{C}\) a.s. and Var(X_i)=Σ, then:
\[
\mathbb{P}(\|\hat\Sigma-\Sigma\| > \varepsilon)\le 2d\exp\left(-\frac{n\varepsilon^2}{2C(C+2\varepsilon/3)}\right)
\]

**Weyl**: if \(\hat\Sigma = \Sigma + E\), then
\[
\max_i |\hat\lambda_i - \lambda_i| \le \|E\|.
\]

Below are helpers to compute empirical covariance, operator norm, and the bounds.


In [ ]:
def empirical_covariance_centered(X: np.ndarray) -> np.ndarray:
    """Assumes X is centered (mean ~ 0). Returns (d,d)."""
    X = np.asarray(X, dtype=float)
    n = X.shape[0]
    return (X.T @ X) / n

def operator_norm(M: np.ndarray) -> float:
    """Spectral/operator norm (largest singular value)."""
    M = np.asarray(M, dtype=float)
    return float(np.linalg.norm(M, ord=2))

def matrix_bernstein_bound_thm11_15(d: int, n: int, eps: float, C: float) -> float:
    """Thm 11.15: 2 d exp( - n eps^2 / (2 C (C + 2 eps/3)) )."""
    if d <= 0 or n <= 0 or eps <= 0 or C <= 0:
        raise ValueError("d,n,eps,C must be positive.")
    denom = 2.0 * C * (C + 2.0*eps/3.0)
    return float(2.0 * d * math.exp(- n * eps * eps / denom))

def weyl_eigenvalue_deviation_bound(E: np.ndarray) -> float:
    """Thm 11.17 bound: max |λhat - λ| <= ||E||."""
    return operator_norm(E)

# Tiny demo: covariance estimation error vs bound (C must be valid a.s. bound; here we clip)
rng = np.random.default_rng(0)
d = 20
n = 4000
X = rng.normal(size=(n, d))
Xc, _ = center_data(X)

# Artificially enforce a.s. bound by clipping norms (so C is valid)
norms = np.linalg.norm(Xc, axis=1, keepdims=True)
Xc_clip = Xc / np.maximum(1.0, norms/5.0)  # clip to norm<=5
C = 25.0  # (sqrt(C)=5)

Sigma_hat = empirical_covariance_centered(Xc_clip)
# True Sigma is approx I for clipped; treat I as reference for demo
E = Sigma_hat - np.eye(d)
eps = operator_norm(E)

print("||Sigma_hat - I|| =", eps)
print("Bernstein bound P(||...|| > eps) <= ", matrix_bernstein_bound_thm11_15(d, n, eps, C))
print("Weyl bound on eigenvalue max deviation:", weyl_eigenvalue_deviation_bound(E))


## 11.6 Reconstruction error risk + excess risk (Lemma 11.19, Thm 11.20)

Define the class of rank-k orthogonal projections:
\[
\mathcal{P}_k = \{\Pi:\mathbb{R}^d\to\mathbb{R}^d \mid \Pi \text{ is orthogonal projection of rank } k\}.
\]

Loss:
\[
L(x,\Pi(x)) = \|x-\Pi(x)\|_2^2
\]
Risk:
\[
R(\Pi) = \mathbb{E}[L(X,\Pi(X))]
\]

Excess risk:
\[
E_k = R(\hat\Pi_k^*) - R(\Pi_k^*).
\]

Lemma 11.19:
\[
E_k \le \sqrt{2k}\,\|\Sigma-\hat\Sigma\|_2
\]

Thm 11.20 (plugging Thm 11.15):
\[
\mathbb{P}(E_k > \varepsilon)\le 2d\exp\left(-\frac{n\varepsilon^2}{4C(C+2\varepsilon/3)\,k}\right)
\]

Below are helpers for these bounds.


In [ ]:
def excess_risk_upper_from_cov_error(k: int, cov_error_op_norm: float) -> float:
    """Lemma 11.19: E_k <= sqrt(2k) * ||Sigma - Sigma_hat||."""
    if k <= 0:
        raise ValueError("k>0.")
    return float(math.sqrt(2.0 * k) * cov_error_op_norm)

def excess_risk_tail_bound_thm11_20(d: int, n: int, eps: float, k: int, C: float) -> float:
    """Thm 11.20 tail bound for E_k."""
    if d <= 0 or n <= 0 or eps <= 0 or k <= 0 or C <= 0:
        raise ValueError("d,n,eps,k,C positive.")
    denom = 4.0 * C * (C + 2.0*eps/3.0) * k
    return float(2.0 * d * math.exp(- n * eps * eps / denom))

# Demo numbers
print("Example tail bound:", excess_risk_tail_bound_thm11_20(d=64, n=2000, eps=0.5, k=10, C=25.0))


## ✅ Function index (copy/paste friendly)

### Random Projection + JL
- `random_projection_bound_thm11_1(k, eps)`
- `jl_required_k(n_points, eps)` + `jl_success_prob_lower(n_points)`
- `sample_subgaussian_matrix(d, k, a, rng)`
- `random_projection_map(X, R, scale_by_sqrt_k=True)`
- `pairwise_distances(X)` + `relative_distance_errors(X, Y)`

### SVD / power method (Lemma 11.7, Sec 11.2.1)
- `power_method_top_eigenvector(B, n_iter, tol, seed)`
- `top_k_eigenvectors_deflation(B, k, seed)`
- `svd_from_ata(A, k, seed)`

### PCA + compression
- `center_data(X)`
- `pca_fit(X, k)` + `pca_transform(X, pca)` + `pca_inverse_transform(Z, pca)`
- `rank_k_approximation(X, k)`
- `reconstruction_error_frobenius(X, Xk)`
- `reconstruction_error_from_singular_values(S, k)`
- `explained_variance_ratio_from_singular_values(S, k)`

### Anomaly detection (Sec 11.4.3)
- `pca_anomaly_detector_fit(X_train, k, q)`
- `pca_anomaly_detector_predict(det, X)`

### Theory bounds (Thm 11.15, 11.17, 11.19, 11.20)
- `empirical_covariance_centered(X_centered)`
- `operator_norm(M)`
- `matrix_bernstein_bound_thm11_15(d, n, eps, C)`
- `weyl_eigenvalue_deviation_bound(E)`
- `excess_risk_upper_from_cov_error(k, cov_error_op_norm)`
- `excess_risk_tail_bound_thm11_20(d, n, eps, k, C)`
